<DIV ALIGN=CENTER>

# Introduction to Social Media: Twitter
## Professor Robert J. Brunner
  
</DIV>  
-----
-----


## Introduction

In this IPython Notebook, we explore 

OAUTH

Twitter client 

Process twitter information. SHow Tweet JSON.

Discuss rate limits

Sentiment analysis?

-----


In [1]:
# Set up Notebook

% matplotlib inline

# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# We do this to ignore several specific Pandas warnings
import warnings
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.4/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


-----

## Student Activity

In the preceding cells, we used . Now that you
have run the Notebook, go back and make the following changes to see how
the results change.

1. Change the ?
2. Change the ?
3. Try making a ?

Finally, try applying a (discuss in class forums?

-----